# Get data

In [ ]:
import numpy as np
import yfinance as yf
import tensorflow as tf
import matplotlib.pyplot as plt

from scipy.optimize import fmin
from scipy.special import lambertw
from scipy.stats import kurtosis, zscore

from tensorflow import keras
from gan import GAN

In [ ]:
raw_data = yf.download('^GSPC','2009-05-01','2018-12-31')

In [ ]:
def delta_GMM(x, gamma_0):
  delta_0 = 1 / 66 * (np.sqrt(66 * gamma_0 - 162) - 6)

  def f(delta):
    u_delta = W_delta(x, delta)
    gamma = kurtosis(u_delta)
    return np.abs(gamma - gamma_0)

  res = fmin(f, delta_0)
  return res.xopt


def W_delta(z, delta):
  return np.sign(z) * np.sqrt(np.real(lambertw(delta * z ** 2)) / delta)


def IGMM(y, tol=1e-6, n_max=1000):
  tau_prev = np.array([0, 0, 0])
  gamma = kurtosis(y)
  delta = 1 / 66 * (np.sqrt(66 * gamma - 162) - 6)
  tau = np.array([np.median(y), np.std(y) * np.power(1 - 2 * delta, 0.75), delta])
  k = 0
  while np.linalg.norm(tau - tau_prev) > tol and k < n_max:
    tau_prev = tau
    z = (y - tau[0]) / tau[1]
    tau[2] = delta_GMM(z)
    x = W_delta((z - tau[0]) / tau[1], tau[2]) * tau[1] + tau[0]
    tau[0], tau[1] = np.mean(x), np.std(x)
    k = k + 1
  
  if k > n_max:
    raise RuntimeError(f"IGMM did not converge after {n_max} iterations")
  return tau

def rolling_window(array, window_size,freq):
  shape = (array.shape[0] - window_size + 1, window_size)
  strides = (array.strides[0],) + array.strides
  rolled = np.lib.stride_tricks.as_strided(array, shape=shape, strides=strides)
  return rolled[np.arange(0,shape[0],freq)]
    

In [ ]:
# 1) Log returns
logret = np.diff(np.log(raw_data['Close']))
# 2) Standardize
logret_norm = zscore(logret)
# 3) Inverse Lambert W transform
gamma_0 = kurtosis(logret_norm)
tau = IGMM(logret_norm, gamma_0)
processed = W_delta((logret_norm - tau[0]) / tau[1], tau[2])
# 4) Normalize
processed_norm = zscore(processed)
# 5) Rolling window
data = rolling_window(processed_norm, 127, 5)

# Train model

In [ ]:
data = np.expand_dims(data, 2)
data.shape

In [ ]:
batch_size = 16
dataset = tf.data.Dataset.from_tensor_slices(data).batch(batch_size)

In [ ]:
latent_size = 3
hidden_size = 80
output_size = 1
d_train_steps = 5

gan = GAN(latent_size, hidden_size, output_size, d_train_steps)

gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0002),
    loss_fn=keras.losses.BinaryCrossentropy()
)

gan.fit(dataset, epochs=10)

In [ ]:
latent_noise = tf.random.normal(shape=(1, 2433, 3))
fake = gan.generator(latent_noise).numpy().reshape(2433)

In [ ]:
plt.hist(fake, bins=50)
plt.show()

In [ ]:
fake.mean()

In [ ]:
data.mean()